In [1]:
#https://github.com/thushv89/nlp_examples_thushv_dot_com/blob/master/cnn_sentence_classification.ipynb
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
import tensorflow as tf

In [2]:
global dir_name,filenames,num_files

dir_name = 'question-classif-data'
url= 'cogcomp.org/Data/QA/QC/' 

#url= 'http://cogcomp.org/Data/QA/QC/' 

if not os.path.exists(dir_name):
    os.mkdir(dir_name)

filenames = ['trec-train-5500.txt','trec-test.txt']
num_files = 2

for i in range(len(filenames)):
    file_exists = os.path.isfile(os.path.join(dir_name,filenames[i]))
    #assert file_exists

    if not os.path.isfile(os.path.join(dir_name,filenames[i])):
        try:
            print('Downloading training and test datasets')
            urlretrieve(url+"train_5500.label", os.path.join(dir_name,"trec-train-5500.txt"))
            urlretrieve(url+"TREC_10.label", os.path.join(dir_name,"trec-test.txt"))
            print('Download successful')
        except:
            print('Something went wrong while downloading data. \n'+
            'Can you manually download data and place them in the question-classif-data folder?\n'+
            'Train: http://cogcomp.org/Data/QA/QC/train_5500.label as trec-train-5500.txt\n'+
            'Test: http://cogcomp.org/Data/QA/QC/TREC_10.label as trec-test.txt')

print('Files found and verified.')

Files found and verified.


In [3]:
max_sent_length = 0
def read_data(filename):
  global max_sent_length
  questions = []
  labels = []
  with open(filename,'r',encoding='latin-1') as f:        
    for row in f:
        row_str = row.split(":")
        lb,q = row_str[0],row_str[1]
        q = q.lower()
        labels.append(lb)
        questions.append(q.split())        
        if len(questions[-1])>max_sent_length:
            max_sent_length = len(questions[-1])
  return questions,labels


global train_questions,train_labels
global test_questions,test_labels

for i in range(num_files):    
    print('\nProcessing file %s'%os.path.join(dir_name,filenames[i]))
    if i==0:
        train_questions,train_labels = read_data(os.path.join(dir_name,filenames[i]))
        assert len(train_questions)==len(train_labels)
    elif i==1:
        test_questions,test_labels = read_data(os.path.join(dir_name,filenames[i]))
        assert len(test_questions)==len(test_labels)
    for j in range(5):
        print('\tQuestion %d: %s' %(j,train_questions[j]))
        print('\tLabel %d: %s\n'%(j,train_labels[j]))
        
print('Max Sentence Length: %d'%max_sent_length)
print('\nNormalizing all sentences to same length')

for qi,que in enumerate(train_questions):
    for _ in range(max_sent_length-len(que)):
        que.append('PAD')
    assert len(que)==max_sent_length
    train_questions[qi] = que
print('\tTrain questions normalized')
for qi,que in enumerate(test_questions):
    for _ in range(max_sent_length-len(que)):
        que.append('PAD')
    assert len(que)==max_sent_length
    test_questions[qi] = que
print('\tTest questions normalized')  
print('\t\tSample test question: %s',test_questions[0])


Processing file question-classif-data\trec-train-5500.txt
	Question 0: ['manner', 'how', 'did', 'serfdom', 'develop', 'in', 'and', 'then', 'leave', 'russia', '?']
	Label 0: DESC

	Question 1: ['cremat', 'what', 'films', 'featured', 'the', 'character', 'popeye', 'doyle', '?']
	Label 1: ENTY

	Question 2: ['manner', 'how', 'can', 'i', 'find', 'a', 'list', 'of', 'celebrities', "'", 'real', 'names', '?']
	Label 2: DESC

	Question 3: ['animal', 'what', 'fowl', 'grabs', 'the', 'spotlight', 'after', 'the', 'chinese', 'year', 'of', 'the', 'monkey', '?']
	Label 3: ENTY

	Question 4: ['exp', 'what', 'is', 'the', 'full', 'form', 'of', '.com', '?']
	Label 4: ABBR


Processing file question-classif-data\trec-test.txt
	Question 0: ['manner', 'how', 'did', 'serfdom', 'develop', 'in', 'and', 'then', 'leave', 'russia', '?']
	Label 0: DESC

	Question 1: ['cremat', 'what', 'films', 'featured', 'the', 'character', 'popeye', 'doyle', '?']
	Label 1: ENTY

	Question 2: ['manner', 'how', 'can', 'i', 'find', 

In [4]:
def build_dataset(questions):
    words = []
    data_list = []
    count = []
    for d in questions:
        words.extend(d)
    print('%d Words found.'%len(words))    
    print('Found %d words in the vocabulary. '%len(collections.Counter(words).most_common()))
    count.extend(collections.Counter(words).most_common())
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
        
    for d in questions:
        data = list()
        for word in d:
            index = dictionary[word]        
            data.append(index)
            
        data_list.append(data)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data_list, count, dictionary, reverse_dictionary

global data_list, count, dictionary, reverse_dictionary

all_questions = list(train_questions)
all_questions.extend(test_questions)

all_question_ind, count, dictionary, reverse_dictionary = build_dataset(all_questions)

print('All words (count)', count[:5])
print('0th entry in dictionary: %s',reverse_dictionary[0])
print('\nSample data') 
print(all_question_ind[0])
print(all_question_ind[1])

print('\nVocabulary size: ',len(dictionary))
vocabulary_size = len(dictionary)

print('\nTrain size: ',len(train_questions))
print('Test size: ',len(test_questions))

226176 Words found.
Found 8917 words in the vocabulary. 
All words (count) [('PAD', 161576), ('?', 5790), ('the', 4036), ('what', 3725), ('is', 1959)]
0th entry in dictionary: %s PAD

Sample data
[33, 9, 20, 3622, 2268, 6, 19, 506, 844, 1019, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[37, 3, 845, 630, 2, 161, 1277, 3623, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Vocabulary size:  8917

Train size:  5452
Test size:  500


In [5]:
all_question_ind[0]

[33,
 9,
 20,
 3622,
 2268,
 6,
 19,
 506,
 844,
 1019,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [6]:
reverse_dictionary

{0: 'PAD',
 1: '?',
 2: 'the',
 3: 'what',
 4: 'is',
 5: 'of',
 6: 'in',
 7: 'a',
 8: 'ind',
 9: 'how',
 10: 'other',
 11: "'s",
 12: 'was',
 13: 'who',
 14: 'to',
 15: ',',
 16: 'def',
 17: 'are',
 18: 'for',
 19: 'and',
 20: 'did',
 21: 'does',
 22: 'count',
 23: '``',
 24: "''",
 25: 'do',
 26: 'name',
 27: 'many',
 28: 'on',
 29: 'desc',
 30: 'where',
 31: 'country',
 32: 'date',
 33: 'manner',
 34: 'city',
 35: 'first',
 36: 'when',
 37: 'cremat',
 38: 'reason',
 39: 'i',
 40: 'gr',
 41: 'you',
 42: 'can',
 43: 'world',
 44: 'state',
 45: 'from',
 46: 'which',
 47: 'animal',
 48: 'u.s.',
 49: 'that',
 50: 'an',
 51: 'most',
 52: 'by',
 53: 'food',
 54: 'with',
 55: 'as',
 56: 'have',
 57: 'has',
 58: 'why',
 59: 'dismed',
 60: 'there',
 61: 'termeq',
 62: 'it',
 63: '.',
 64: 'color',
 65: 'money',
 66: 'called',
 67: 'year',
 68: 'people',
 69: 'at',
 70: 'were',
 71: 'period',
 72: 'get',
 73: 'be',
 74: 'president',
 75: 'sport',
 76: 'find',
 77: 'word',
 78: 'his',
 79: 'mean

In [7]:
batch_size = 16
sent_length = 40
num_classes = 6
all_labels = ['NUM','LOC','HUM','DESC','ENTY','ABBR'] # All the types of question that are in the dataset

class BatchGenerator(object):
    def __init__(self,batch_size,questions,labels):
        self.questions = questions
        self.labels = labels
        self.text_size = len(questions)
        self.batch_size = batch_size
        self.data_index = 0
        assert len(self.questions)==len(self.labels)
        
    def generate_batch(self):
        global sent_length,num_classes
        global dictionary
        inputs = np.zeros((self.batch_size,sent_length,vocabulary_size),dtype=np.float32)
        labels_ohe = np.zeros((self.batch_size,num_classes),dtype=np.float32)
        
        if self.data_index + self.batch_size >= self.text_size:
            self.data_index = 0
            
        for qi,que in enumerate(self.questions[self.data_index:self.data_index+self.batch_size]):
            for wi,word in enumerate(que):
                #q_arr[qi*batch_size + wi] = dictionary[word]
                inputs[qi,wi,dictionary[word]] = 1.0
            
            #print(self.data_index,qi,len(self.labels))
            labels_ohe[qi,all_labels.index(self.labels[self.data_index + qi])] = 1.0
            #print('%s'%([w for w in que]),self.labels[self.data_index+qi])            
            
        self.data_index = (self.data_index + self.batch_size)%self.text_size
            
        return inputs,labels_ohe
    
    def return_index(self):
        return self.data_index
    
sample_gen = BatchGenerator(batch_size,train_questions,train_labels)
sample_batch_inputs,sample_batch_labels = sample_gen.generate_batch()
sample_batch_inputs_2,sample_batch_labels_2 = sample_gen.generate_batch()
assert np.all(np.asarray([dictionary[w] for w in train_questions[0]],dtype=np.int32) 
              == np.argmax(sample_batch_inputs[0,:,:],axis=1))
print('Sample batch labels')
print(np.argmax(sample_batch_labels,axis=1))
print(np.argmax(sample_batch_labels_2,axis=1))

C:\Users\Catherine - Current\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


AssertionError: 

In [8]:
batch_size = 16

# inputs and labels
sent_inputs = tf.placeholder(shape=[batch_size,sent_length,vocabulary_size],dtype=tf.float32,name='sentence_inputs')
sent_labels = tf.placeholder(shape=[batch_size,num_classes],dtype=tf.float32,name='sentence_labels')

# 3 filters with different context window sizes (3,5,7)
# Each of this filter spans the full one-hot-encoded length of each word and the context window width
w1 = tf.Variable(tf.truncated_normal([3,vocabulary_size,1],stddev=0.02,dtype=tf.float32),name='weights_1')
b1 = tf.Variable(tf.random_uniform([1],0,0.01,dtype=tf.float32),name='bias_1')

w2 = tf.Variable(tf.truncated_normal([5,vocabulary_size,1],stddev=0.02,dtype=tf.float32),name='weights_2')
b2 = tf.Variable(tf.random_uniform([1],0,0.01,dtype=tf.float32),name='bias_2')

w3 = tf.Variable(tf.truncated_normal([7,vocabulary_size,1],stddev=0.02,dtype=tf.float32),name='weights_3')
b3 = tf.Variable(tf.random_uniform([1],0,0.01,dtype=tf.float32),name='bias_3')

# Calculate the output for all the filters with a stride 1
h1_1 = tf.nn.tanh(tf.nn.conv1d(sent_inputs,w1,stride=1,padding='SAME') + b1)
h1_2 = tf.nn.tanh(tf.nn.conv1d(sent_inputs,w2,stride=1,padding='SAME') + b2)
h1_3 = tf.nn.tanh(tf.nn.conv1d(sent_inputs,w3,stride=1,padding='SAME') + b3)

# This is doing the max pooling. Thereare two options to do the max pooling
# 1. Use tf.nn.max_pool operation on a tensor made by concatenating h1_1,h1_2,h1_3 and converting that tensor to 4D
# (Because max_pool takes a tensor of rank >= 4 )
# 2. Do the max pooling separately for each filter output and combine them using tf.concat 
# (this is the one used in the code)

h2_1 = tf.reduce_max(h1_1,axis=1)
h2_2 = tf.reduce_max(h1_2,axis=1)
h2_3 = tf.reduce_max(h1_3,axis=1)

h2 = tf.concat([h2_1,h2_2,h2_3],axis=1)
h2_shape = h2.get_shape().as_list()

# Weights and bias of the output layer
w_fc1 = tf.Variable(tf.truncated_normal([h2_shape[1],num_classes],stddev=0.005,dtype=tf.float32),name='weights_fulcon_1')
b_fc1 = tf.Variable(tf.random_uniform([num_classes],0,0.01,dtype=tf.float32),name='bias_fulcon_1')

# since h2 is 2d [batch_size,output_width] reshaping the output is not required as it usually do in CNNs
logits = tf.matmul(h2,w_fc1) + b_fc1

predictions = tf.argmax(tf.nn.softmax(logits),axis=1)

# Loss (Cross-Entropy)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=sent_labels,logits=logits))

# Momentum Optimizer
optimizer = tf.train.MomentumOptimizer(learning_rate=0.01,momentum=0.9).minimize(loss)

In [9]:
# With filter widths [3,5,7] the algorithm achieves around ~90% accuracy on test dataset (50 epochs). 

session = tf.InteractiveSession()

batch_size = 16
num_steps = 50
tf.global_variables_initializer().run()
train_gen = BatchGenerator(batch_size,train_questions,train_labels)
test_gen = BatchGenerator(batch_size,test_questions,test_labels)

test_interval = 1

# Get accuracy 
def accuracy(labels,preds):
    return np.sum(np.argmax(labels,axis=1)==preds)/labels.shape[0]

print('Initialized\n')


for step in range(num_steps):
    avg_loss = []
    for tr_i in range((len(train_questions)//batch_size)-1):
        tr_inputs, tr_labels = train_gen.generate_batch()
        
        l,_ = session.run([loss,optimizer],feed_dict={sent_inputs: tr_inputs, sent_labels: tr_labels})
        avg_loss.append(l)
        
    print('Train Loss at Epoch %d: %.2f'%(step,np.mean(avg_loss)))
    test_accuracy = []
    if (step+1)%test_interval==0:        
        for ts_i in range((len(test_questions)-1)//batch_size):
            ts_inputs,ts_labels = test_gen.generate_batch()
            preds = session.run(predictions,feed_dict={sent_inputs: ts_inputs, sent_labels: ts_labels})
            test_accuracy.append(accuracy(ts_labels,preds))
            
        print('Test accuracy at Epoch %d: %.3f'%(step,np.mean(test_accuracy)*100.0))

Initialized

Train Loss at Epoch 0: 1.68
Test accuracy at Epoch 0: 13.105
Train Loss at Epoch 1: 1.57
Test accuracy at Epoch 1: 43.952
Train Loss at Epoch 2: 0.92
Test accuracy at Epoch 2: 75.202
Train Loss at Epoch 3: 0.60
Test accuracy at Epoch 3: 79.032
Train Loss at Epoch 4: 0.45
Test accuracy at Epoch 4: 83.065
Train Loss at Epoch 5: 0.33
Test accuracy at Epoch 5: 92.540
Train Loss at Epoch 6: 0.22
Test accuracy at Epoch 6: 93.750
Train Loss at Epoch 7: 0.16
Test accuracy at Epoch 7: 94.153
Train Loss at Epoch 8: 0.13
Test accuracy at Epoch 8: 93.952
Train Loss at Epoch 9: 0.11
Test accuracy at Epoch 9: 93.952
Train Loss at Epoch 10: 0.10
Test accuracy at Epoch 10: 95.363
Train Loss at Epoch 11: 0.09
Test accuracy at Epoch 11: 95.968
Train Loss at Epoch 12: 0.08
Test accuracy at Epoch 12: 95.968
Train Loss at Epoch 13: 0.07
Test accuracy at Epoch 13: 95.968
Train Loss at Epoch 14: 0.07
Test accuracy at Epoch 14: 95.968
Train Loss at Epoch 15: 0.06
Test accuracy at Epoch 15: 96.169

In [10]:
print('done')

done
